In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports

In [6]:
import os
import pandas as pd
import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx
import json

In [14]:
import threading
threading.activeCount()

7

# Load

In [7]:
PATH1 = os.path.join(os.getcwd(), 'data/100k_a.csv')
PATH2 = 'data/FULL_a.csv.gz'

In [8]:
df = pd.read_csv(PATH1, names=['user_id', 'stream_id', 'streamer_name', 'start_time', 'stop_time'])

In [10]:
# df_full = pd.read_csv(PATH2, names=['user_id', 'stream_id', 'streamer_name', 'start_time', 'stop_time'])

In [11]:
# df2 = df_full[:10000000]

NameError: name 'df_full' is not defined

# Basic logging

In [12]:
def df_stats(df):
    for col in df.columns:
        # print(f"{col} - # unique: {len(set(df[col]))}, # total: ")
        num_unique = len(set(df[col]))
        print(f"# of unique {col}: {num_unique}")
        print(f"Average data per unique: {len(df) / num_unique}")
        print("\n")
        

In [13]:
df_stats(df)

# of unique user_id: 100000
Average data per unique: 30.51733


# of unique stream_id: 739991
Average data per unique: 4.124013670436533


# of unique streamer_name: 162625
Average data per unique: 18.765460415065334


# of unique start_time: 6148
Average data per unique: 496.37817176317503


# of unique stop_time: 6148
Average data per unique: 496.37817176317503




In [47]:
# df_stats(df2)

# of unique user_id: 339294
# of unique stream_id: 1449583
# of unique streamer_name: 278411
# of unique start_time: 6148
# of unique stop_time: 6148


In [48]:
# df_stats(df_full)

# of unique user_id: 15524309
# of unique stream_id: 6161666
# of unique streamer_name: 790100
# of unique start_time: 6148
# of unique stop_time: 6148


# Creating Graph
Bipartite graph of users + streamers
Two streamers are connected if a user visited both of them

# Network Analysis (unweighted)

In [15]:
from collections import defaultdict

In [16]:
# creating edge list
edge_list = set()
for i, row in df.iterrows():
    user = row['user_id']
    streamer_name = row['streamer_name']
    edge_list.add((user, streamer_name))
    if i % 100000 == 0:
        print(f"{i} / {len(df)}")

0 / 3051733
100000 / 3051733
200000 / 3051733
300000 / 3051733
400000 / 3051733
500000 / 3051733
600000 / 3051733
700000 / 3051733
800000 / 3051733
900000 / 3051733
1000000 / 3051733
1100000 / 3051733
1200000 / 3051733
1300000 / 3051733
1400000 / 3051733
1500000 / 3051733


KeyboardInterrupt: 

In [23]:
# Graph
G = nx.Graph()
G.add_edges_from(edge_list)

In [25]:
len(G)

262625

In [26]:
# Projection
user_set = set(df['user_id'])
streamer_name_set = set(df['streamer_name'])

In [27]:
from networkx.algorithms import bipartite

In [28]:
projected = bipartite.projected_graph(G, streamer_name_set)

In [32]:
len(projected.edges)

11048847

In [35]:
partition = community_louvain.best_partition(projected)

# 364 different assignnments

In [39]:
len(set(partition.values()))

364

In [43]:
# save the partition
with open('clusters.json', 'w') as fp:
    json.dump(partition, fp)

In [30]:
B = nx.path_graph(4)

In [35]:
B.edges

EdgeView([(0, 1), (1, 2), (2, 3)])

# Weighted Clustering

In [45]:
from collections import defaultdict

In [46]:
# creating edge list
edge_list = defaultdict(int)
for i, row in df.iterrows():
    user = row['user_id']
    streamer_name = row['streamer_name']
    edge_list[(user, streamer_name)] += 1
    if i % 100000 == 0:
        print(f"{i} / {len(df)}")

0 / 3051733
100000 / 3051733
200000 / 3051733
300000 / 3051733
400000 / 3051733
500000 / 3051733
600000 / 3051733
700000 / 3051733
800000 / 3051733
900000 / 3051733
1000000 / 3051733
1100000 / 3051733
1200000 / 3051733
1300000 / 3051733
1400000 / 3051733
1500000 / 3051733
1600000 / 3051733
1700000 / 3051733
1800000 / 3051733
1900000 / 3051733
2000000 / 3051733
2100000 / 3051733
2200000 / 3051733
2300000 / 3051733
2400000 / 3051733
2500000 / 3051733
2600000 / 3051733
2700000 / 3051733
2800000 / 3051733
2900000 / 3051733
3000000 / 3051733


In [47]:
# Graph
G = nx.Graph()
G.add_edges_from(edge_list)